# Coupled Cluster Calculations of Infinite Nuclear Matter in the Complete Basis Limit Using Bayesian Machine Learning
## Source Code

Date Created: August 17, 2024

Last Modified: August 17, 2024

## Imports

In [24]:
%%time
##############################
##          IMPORTS         ##
##############################
# THIRD PARTY IMPORTS
# For arrays
import numpy as np
# For importing the data set
import pandas as pd
# For graphing
import matplotlib.pyplot as plt
import glob
from sklearn.gaussian_process.kernels import ConstantKernel, WhiteKernel, RationalQuadratic
from sklearn.gaussian_process import GaussianProcessRegressor as GP

##############################
##        DIRECTORIES       ##
##############################
# Directories to find data files and save images
data_dir = 'raw_data/'
images_dir = 'img/'

CPU times: user 9.17 ms, sys: 8.33 ms, total: 17.5 ms
Wall time: 18.6 ms


## Importing and Formatting Data

### CCD PNM

In [18]:
ccd_pnm_data_dir = data_dir+"CCD_PNM/"
ccd_pnm_data = []
for file in glob.glob(ccd_pnm_data_dir+"*.csv"):
    df = pd.read_csv(file)
    ccd_pnm_data.append(df.to_numpy())

### CCD SNM

In [19]:
ccd_snm_data_dir = data_dir+"CCD_SNM/"
ccd_snm_data = []
for file in glob.glob(ccd_snm_data_dir+"*.csv"):
    df = pd.read_csv(file)
    ccd_snm_data.append(df.to_numpy())

### CCD(T) PNM

In [20]:
ccdt_pert_pnm_data_dir = data_dir+"CCDT_Pert_PNM/"
ccdt_pert_pnm_data = []
for file in glob.glob(ccdt_pert_pnm_data_dir+"*.csv"):
    df = pd.read_csv(file)
    ccdt_pert_pnm_data.append(df.to_numpy())

### CCD(T) SNM

In [21]:
ccdt_pert_snm_data_dir = data_dir+"CCDT_Pert_SNM/"
ccdt_pert_snm_data = []
for file in glob.glob(ccdt_pert_snm_data_dir+"*.csv"):
    df = pd.read_csv(file)
    ccdt_pert_snm_data.append(df.to_numpy())

## Figure 1

In [23]:
##############################
##      GET STATES DICT     ##
##############################    
def get_states_dict ():
    """
    Inputs:
        None.
    Returns:
        states_dict (a dictionary): the conversion between shell number and number of 
            single particle states.
    Dictionary thats converts between number of shells to number of single particle states
    for the first 178 shells.  
    """
    states_dict = {
        1:2, 2:14, 3:38, 4:54, 5:66, 6:114, 7:162, 8:186, 9:246, 10:294, 
        11:342, 12:358, 13:406, 14:502, 15:514, 16:610, 17:682, 18:730, 
        19:778, 20:874, 21:922, 22:970, 23:1030, 24:1174, 25:1238, 26:1382, 
        27:1478, 28:1502,29:1598, 30:1694, 31:1790, 32:1850, 33:1898, 
        34:2042, 35:2090, 36:2282, 37:2378, 38:2426, 39:2474, 40:2618, 
        41:2714, 42:2730, 43:2838, 44:3006, 45:3102, 46:3150, 47:3294, 
        48:3486, 49:3582, 50:3678, 51:3726, 52:3870, 53:4014, 54:4206, 
        55:4218, 56:4410, 57:4602, 58:4650, 59:4746,  60:4938, 61:5034, 
        62:5106, 63:5202, 64:5442, 65:5554, 66:5602, 67:5794, 68:5890,
        69:5938, 70:6142, 71:6238, 72:6382, 73:6478, 74:6574, 75:6814,
        76:6862, 77:7150, 78:7390, 79:7486, 80:7582, 81:7774, 82:7822,
        83:7918, 84:8134, 85:8278, 86:8338, 87:8674, 88:8770, 89:8914,
        90:9106 ,91:9250, 92:9394, 93:9458, 94:9602, 95:9890, 96:10082,
        97:10274, 98:10370, 99:10514, 100:10754, 101:10898, 102:10994,
        103:11138, 104:11330, 105:11378, 106:11618, 107:11810, 108:11834,
        109:12074, 110:12122, 111:12266, 112:12362, 113:12458, 114:12698,
        115:12794, 116:12938, 117:13034, 118:13130, 119:13226, 120:13322,
        121:13418, 122:13466, 123:13610, 124:13802, 125:13818, 126:14058,
        127:14202, 128:14298, 129:14490, 130:14586, 131:14682, 132:14778,
        133:14970, 134:15042, 135:15090, 136:15186, 137:15378, 138:15522,
        139:15618, 140:15714, 141:15762, 142:15810, 143:15906, 144:16002,
        145:16050, 146:16098, 147:16146, 148:16242, 149:16386, 150:16482,
        151:16674, 152:16722, 153:16818, 154:16914, 155:16930, 156:17026,
        157:17122, 158:17218, 159:17338, 160:17386, 161:17434, 162:17530,
        163:17626, 164:17674, 165:17770, 166:17818, 167:17914, 168:17962,
        169:18010, 170:18058, 171:18154, 172:18202, 173:18218, 174:18314,
        175:18362, 176:18410, 177:18458, 178:18506}
        
    return states_dict

### Left

### Right

## Figure 2

In [22]:
##############################
##  FORMAT SEQUENTIAL DATA  ##
##############################
def format_sequential_data (y, seq=2):
    """
        Inputs:
            y (a list or NumPy array): the y values of a data set
            seq (an int): the length of the sequence.  Default value is 2
        Returns:
            inputs (a list): the inputs for a machine learning model using 
                sequential data formatting
            outputs (a list): the outputs for a machine learning model using
                sequential data formatting              
        Formats a given list or array in sequential formatting using the 
        given sequence lenght.  Default sequence length is two.

        Explanation of sequential formatting:
        Typically data points of the form (x,y) are used to train a machine
        learning model.  This teaches the model the relationship between the
        x data and the y data in the training range.  This model works well 
        for interpolation, but not so well for extrapolation.  A better data
        model for extrapolation would be one that learns the patterns in the y
        data to better guess what y value should come next.  Therefore, this 
        method formats the data in a sequential pattern so that the points are
        of the form ((y1, y2, ..., yn), yn+1) where n is the lenght of the 
        sequence (seq).
    """
    # Make sure seq is an int
    assert isinstance(seq, int)
    # Set up the input and output lists
    inputs = []
    outputs = []
    # Cycle through the whole y list/array and separate the points into 
    # sequential format
    for i in range(0, len(y)-seq):
        inputs.append(y[i:i+seq])
        outputs.append(y[i+seq])
    # Return the input and output lists.  NOTE: the data type of the return 
    # values is LIST
    return inputs, outputs  
####################################
## SEQUENTIAL EXTRAPOLATE SKLEARN ##
####################################
def sequential_extrapolate_sklearn (R, y_train, num_points, seq):
    """
        Inputs:
            R (an object): A trained Scikit-Learn regression model
            y_train (a list): the y component of the training data set, unformatted
            num_points (an int): the number of points to be in the extrapolated data set
            seq (an int): the SRE length of sequence that R was trained with
        Returns:
            y_test (a list): the extrapolated data set
            y_std (a list): the uncertainity on each point in the extrapoalted set
        Performs SRE using a trained Scikit-Learn regression model.
    """
    # Make sure inputs are of the proper type
    assert isinstance(num_points, int)
    assert isinstance(seq, int)

    # Add the training data to the extrapolated data set and no uncertainities for the
    # training data
    y_test = y_train.copy()
    y_std = np.zeros(len(y_train)).tolist()

    # Extrapolate until enough data points have been predicted
    while len(y_test) < num_points:
        next_test = y_test[-seq:]
        point,std = R.predict([next_test],return_std=True)
        y_test.append(point[0])
        y_std.append(std[0])

    # Return the predicted data set and uncertainities
    return y_test,y_std    
def perform_extrapolations (data_sets,dim=4, start_dim=1, seq=1, n = 4,non_seq=False):
    results = []
    # For each data set
    for A in data_sets:
        # Separate the columns
        shells = A[:,0]
        particles = A[:,1]
        matter_type = A[:,2]
        density = A[:,3]
        reference_E = A[:,4]
        mbpt2_E = A[:,5]
        ccd_E = A[:,6]
        ccd_times = A[:,7]
        # Calculate the correlation energies (total not per particle)
        mbpt2_correlation_E = (mbpt2_E - reference_E)*particles[0]
        ccd_correlation_E = (ccd_E - reference_E)*particles[0]
        # Seperate the final values (the test data), and the training data
        final_cc = ccd_correlation_E[-1]
        final_mbpt =mbpt2_correlation_E[-1]
        cc_seq = ccd_correlation_E[start_dim:dim]
        mbpt_seq = mbpt2_correlation_E[start_dim:dim]
        # Separate the specified points for the training data, without the first one
        # which would yield 0/0 with division
        training_data = cc_seq/mbpt_seq
        training_data = training_data.tolist()
        # Format the training data in sequential series formatting
        x_train, y_train = format_sequential_data (training_data, seq=seq)
        # Set up kernel function
        kernel = ConstantKernel()*RationalQuadratic()+WhiteKernel()
        # Set up Gaussian Process algorithm and train it
        R = GP(kernel=kernel, alpha=np.std(training_data)**n,n_restarts_optimizer=5)
        #R = BayesianRidge(n_iter=10000,tol=1e-5)
        R.fit(x_train, y_train)
        # Feed the trained GP algorithm and data into the SRE method
        ypred,ystd = sequential_extrapolate_sklearn(R, training_data, 50, seq=seq)
        # Separate the last prediction as the final converged slope
        final_slope = ypred[-1]
        # Predict the CC correlation energy at the long range point
        cc_prediction = final_slope*final_mbpt
        # Calculate the uncertainity on the CC correlation energy prediction
        slope_uncertainity = ystd[-1]
        #print(slope_uncertainity)
        cc_prediction_uncertainity = slope_uncertainity*final_mbpt
        results.append([cc_prediction, final_cc, np.sum(ccd_times[start_dim:dim]), ccd_times[-1], ccd_correlation_E[dim-1], cc_prediction_uncertainity])
    return results

### Left

### Right

## Figure 3